<a href="https://colab.research.google.com/github/Hanh42/AI---UEH-K50/blob/main/App_RIDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CÀI ĐẶT THƯ VIỆN (chạy 1 lần trong notebook)
%pip install -q osmnx folium geopy scikit-fuzzy ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.7 MB/s eta 0:00:00


In [2]:


import folium
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from geopy.distance import geodesic
import random
import osmnx as ox
import networkx as nx
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# ====================== 20 điểm du lịch TP.HCM ======================
tourist_spots = {
    "Chợ Bến Thành": (10.7721, 106.6983),
    "Dinh Độc Lập": (10.7798, 106.6950),
    "Nhà thờ Đức Bà": (10.7797, 106.6992),
    "Bưu điện trung tâm": (10.7791, 106.6984),
    "Phố đi bộ Nguyễn Huệ": (10.7770, 106.7000),
    "Bảo tàng TP.HCM": (10.7787, 106.6950),
    "Thảo Cầm Viên Sài Gòn": (10.7790, 106.7041),
    "Bitexco Tower": (10.7711, 106.7043),
    "Cầu Mống": (10.7767, 106.7016),
    "Nhà hát TP": (10.7794, 106.6990),
    "Bảo tàng Mỹ thuật": (10.7790, 106.6981),
    "Đường sách Nguyễn Văn Bình": (10.7756, 106.7012),
    "Ngã tư Hàng Xanh": (10.7901, 106.7062),
    "Công viên 23/9": (10.7755, 106.6960),
    "Vincom Center": (10.7765, 106.7006),
    "Saigon Centre": (10.7733, 106.7033),
    "Bến Nhà Rồng": (10.7629, 106.6983),
    "Phú Mỹ Hưng": (10.7297, 106.7090),
    "Cầu Thủ Thiêm": (10.7792, 106.7180),
    "Suối Tiên": (10.8514, 106.7991)
}

# ============================ THEME & CSS ============================
PRIMARY = "#10b981"   # xanh lá chủ đạo kiểu Xanh SM
PRIMARY_DARK = "#0e9f6e"
SECONDARY = "#0ea5e9" # xanh dương phụ trợ
ACCENT = "#f59e0b"    # vàng nhấn
BG_SOFT = "#f0fdf4"   # nền nhạt
TXT = "#0f172a"       # chữ đậm

APP_HEADER = f"""
<div style="background: linear-gradient(90deg, {PRIMARY}, {SECONDARY}); padding:16px; text-align:center; border-radius:14px; margin-bottom:10px; box-shadow: 0 6px 18px rgba(0,0,0,.12);">
  <h1 style="color:white; font-family:Arial Black; margin:0; letter-spacing:.5px;">🚖 RIDO</h1>
  <div style="color:white; font-family:Arial; opacity:.95;">smart drive, smart life</div>
</div>
"""

CARD_WRAP_START = f"""
<div style="display:grid; grid-template-columns: 1.1fr .9fr; gap:14px;">
  <div style="background:{BG_SOFT}; border:2px solid {PRIMARY}; padding:14px; border-radius:16px; box-shadow:0 6px 16px rgba(16,185,129,.15);">
    <h3 style="margin:6px 0 10px 0; color:{PRIMARY_DARK}; font-family:Arial;">📍 Đặt chuyến</h3>
"""

CARD_WRAP_MID_END = f"""
  </div>
  <div style="background:#ffffff; border:2px solid {SECONDARY}; padding:14px; border-radius:16px; box-shadow:0 6px 16px rgba(14,165,233,.15);">
    <h3 style="margin:6px 0 10px 0; color:{SECONDARY}; font-family:Arial;">🧾 Thông tin chuyến đi</h3>
    <div id="fare_card"></div>
  </div>
</div>
"""

ALERT_HTML = lambda msg: f"""
<div style=\"background:#fff7ed;border:2px solid {ACCENT};color:#9a3412;padding:12px;border-radius:12px; margin-bottom:12px; box-shadow:0 4px 10px rgba(245,158,11,.15)\">⚠️ {msg}</div>
"""

# ====================== Widgets lựa chọn ======================
start_selector = widgets.Dropdown(options=list(tourist_spots.keys()), description='Điểm đi:', layout=widgets.Layout(width='100%'))
end_selector   = widgets.Dropdown(options=list(tourist_spots.keys()), description='Điểm đến:', layout=widgets.Layout(width='100%'))
vehicle_selector = widgets.ToggleButtons(options=[('🚗 Ô tô', 'Ô tô'), ('🛵 Xe máy', 'Xe máy')], description='Phương tiện:', button_style='success')

# 5 mức giao thông & thời tiết (chữ + icon) → map về thang 0–4
traffic_levels = {
    "Rất thấp 🌐 ": 0,
    "Thấp 📢": 1,
    "Trung bình 🚦": 2,
    "Cao 🚧": 3,
    "Rất cao ‼️": 4,
}
weather_levels = {
    "Trời đẹp ☀️": 0,
    "Ít mưa 🌦️": 1,
    "Mưa 🌧️": 2,
    "Mưa lớn ⛈️": 3,
    "Bão 🌫": 4,
}

traffic_selector = widgets.RadioButtons(options=list(traffic_levels.keys()), description='Giao thông:')
weather_selector = widgets.RadioButtons(options=list(weather_levels.keys()), description='Thời tiết:')

compute_button = widgets.Button(description="Tính tuyến & báo giá", button_style='success', icon='check')
reset_button   = widgets.Button(description="Làm mới", button_style='warning', icon='refresh')

fare_display = widgets.HTML(value="<i>Chưa có dữ liệu. Hãy chọn điểm đi/đến và bấm tính.</i>")
alert_box = widgets.HTML(value="")

# ====================== Fuzzy logic (5 mức traffic & weather) ======================
# Distance vẫn dùng 3 mức: short/medium/long (đủ thực tế), traffic & weather 5 mức chữ

distance = ctrl.Antecedent(np.arange(0, 20001, 1), 'distance')
traffic  = ctrl.Antecedent(np.arange(0, 5, 1), 'traffic')
weather  = ctrl.Antecedent(np.arange(0, 5, 1), 'weather')
fare     = ctrl.Consequent(np.arange(0, 220001, 1), 'fare')

# Distance membership (m)
distance['short']  = fuzz.trimf(distance.universe, [0, 0, 5000])
distance['medium'] = fuzz.trimf(distance.universe, [3000, 9000, 14000])
distance['long']   = fuzz.trimf(distance.universe, [12000, 20000, 20000])

# Traffic membership (0–4)
traffic['very_low'] = fuzz.trimf(traffic.universe, [0, 0, 1])
traffic['low']      = fuzz.trimf(traffic.universe, [0, 1, 2])
traffic['medium']   = fuzz.trimf(traffic.universe, [1, 2, 3])
traffic['high']     = fuzz.trimf(traffic.universe, [2, 3, 4])
traffic['very_high']= fuzz.trimf(traffic.universe, [3, 4, 4])

# Weather membership (0–4)
weather['clear']  = fuzz.trimf(weather.universe, [0, 0, 1])
weather['light']  = fuzz.trimf(weather.universe, [0, 1, 2])
weather['rain']   = fuzz.trimf(weather.universe, [1, 2, 3])
weather['heavy']  = fuzz.trimf(weather.universe, [2, 3, 4])
weather['storm']  = fuzz.trimf(weather.universe, [3, 4, 4])

# Fare (VND) 5 mức
fare['very_low'] = fuzz.trimf(fare.universe, [0, 0, 60000])
fare['low']      = fuzz.trimf(fare.universe, [40000, 90000, 120000])
fare['medium']   = fuzz.trimf(fare.universe, [100000, 150000, 180000])
fare['high']     = fuzz.trimf(fare.universe, [160000, 190000, 210000])
fare['very_high']= fuzz.trimf(fare.universe, [200000, 220000, 220000])

# Luật suy diễn (heuristic dễ hiểu, ưu tiên rủi ro: bão/kẹt/xa → giá cao)
rules = []
levels_distance = ['short','medium','long']
levels_traffic = ['very_low','low','medium','high','very_high']
levels_weather = ['clear','light','rain','heavy','storm']

for d in levels_distance:
    for t in levels_traffic:
        for w in levels_weather:
            # mapping ưu tiên theo mức rủi ro cao nhất
            if w=='storm' or t=='very_high' or d=='long':
                out = 'very_high'
            elif w=='heavy' or t=='high' or d=='medium':
                out = 'high'
            elif w=='rain' or t=='medium':
                out = 'medium'
            elif t=='low' or w=='light':
                out = 'low'
            else:
                out = 'very_low'
            rules.append(ctrl.Rule(distance[d] & traffic[t] & weather[w], fare[out]))

fare_ctrl = ctrl.ControlSystem(rules)
fare_sim = ctrl.ControlSystemSimulation(fare_ctrl)

# ====================== Sinh xe ngẫu nhiên quanh điểm xuất phát ======================
def generate_random_vehicles_around(start_coords, num_vehicles=24, radius=0.01):
    vehicles = []
    for _ in range(num_vehicles):
        lat_offset = random.uniform(-radius, radius)
        lon_offset = random.uniform(-radius, radius)
        new_lat = start_coords[0] + lat_offset
        new_lon = start_coords[1] + lon_offset
        vehicle_type = random.choice(["Ô tô", "Xe máy"])
        vehicles.append({'lat': new_lat, 'lon': new_lon, 'type': vehicle_type})
    return vehicles

# ====================== HÀM TIỆN ÍCH UI ======================

def format_vnd(n):
    try:
        return f"{int(round(n, 0)):,}".replace(",", ".")
    except Exception:
        return str(n)

# ====================== Tính & hiển thị ======================

def compute_and_display(_):
    alert_box.value = ""
    start_name = start_selector.value
    end_name = end_selector.value
    if start_name == end_name:
        alert_box.value = ALERT_HTML("Điểm đi và điểm đến không được trùng nhau. Hãy chọn lại nhé!")
        return

    start_coords = tourist_spots[start_name]
    end_coords = tourist_spots[end_name]
    vehicle_type = vehicle_selector.value

    # Map chữ → số cho fuzzy
    traffic_value = traffic_levels[traffic_selector.value]
    weather_value = weather_levels[weather_selector.value]

    # Tạo danh sách xe xung quanh điểm đi
    vehicle_data = generate_random_vehicles_around(start_coords, 24, 0.01)

    # Chọn xe gần nhất theo loại
    min_distance = float('inf')
    nearest_vehicle = None
    for v in vehicle_data:
        if v['type'] == vehicle_type:
            d = geodesic(start_coords, (v['lat'], v['lon'])).meters
            if d < min_distance:
                min_distance = d
                nearest_vehicle = v

    # Lấy graph đường đi
    if nearest_vehicle is None:
        # fallback nếu ngẫu nhiên không có xe cùng loại (hiếm)
        nearest_vehicle = vehicle_data[0]

    mid_lat = (nearest_vehicle['lat'] + start_coords[0] + end_coords[0]) / 3
    mid_lon = (nearest_vehicle['lon'] + start_coords[1] + end_coords[1]) / 3

    try:
        G = ox.graph_from_point((mid_lat, mid_lon), dist=5000, network_type='drive')
        node_vehicle = ox.nearest_nodes(G, nearest_vehicle['lon'], nearest_vehicle['lat'])
        node_start = ox.nearest_nodes(G, start_coords[1], start_coords[0])
        node_end = ox.nearest_nodes(G, end_coords[1], end_coords[0])
        route_to_start = nx.shortest_path(G, node_vehicle, node_start, weight='length')
        route_start_to_end = nx.shortest_path(G, node_start, node_end, weight='length')
        coords_to_start = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route_to_start]
        coords_start_to_end = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route_start_to_end]
        total_distance = sum(geodesic(coords_to_start[i], coords_to_start[i+1]).meters for i in range(len(coords_to_start)-1))
        total_distance += sum(geodesic(coords_start_to_end[i], coords_start_to_end[i+1]).meters for i in range(len(coords_start_to_end)-1))
    except Exception:
        # fallback (đo đường thẳng) nếu OSMnx lỗi
        coords_to_start = [(nearest_vehicle['lat'], nearest_vehicle['lon']), start_coords]
        coords_start_to_end = [start_coords, end_coords]
        total_distance = geodesic(coords_to_start[0], coords_to_start[1]).meters + geodesic(coords_start_to_end[0], coords_start_to_end[1]).meters

    # Fuzzy fare
    fare_sim.input['distance'] = float(total_distance)
    fare_sim.input['traffic'] = float(traffic_value)
    fare_sim.input['weather'] = float(weather_value)
    fare_sim.compute()

    base_fare = float(fare_sim.output['fare'])
    final_fare = base_fare * (0.65 if vehicle_type == 'Xe máy' else 1.0)

    # Tốc độ giả lập (km/h)
    speed = 35 if vehicle_type == 'Ô tô' else 30
    travel_time_min = total_distance/1000 / speed * 60

    # Thời gian chờ: phụ thuộc khoảng cách xe + giao thông
    wait_time_min = 3 + (min_distance/1000) * 2 + traffic_value * 1.2
    wait_time_min = min(wait_time_min, 15)

    total_time_min = travel_time_min + wait_time_min

    # Hiển thị thẻ thông tin đẹp như app
    fare_card_html = f"""
    <div style="background:#ffffff; border:1px solid #e2e8f0; padding:14px; border-radius:14px; box-shadow:0 8px 16px rgba(0,0,0,.06); font-family:Arial; color:{TXT}">
      <div style="display:flex; align-items:center; gap:10px; margin-bottom:10px;">
        <div style="background:{PRIMARY}; color:white; padding:8px 10px; border-radius:12px; font-weight:700;">{('🚗' if vehicle_type=='Ô tô' else '🛵')} {vehicle_type}</div>
        <div style="background:{BG_SOFT}; color:{PRIMARY_DARK}; padding:8px 10px; border-radius:12px;">{traffic_selector.value}</div>
        <div style="background:#eff6ff; color:{SECONDARY}; padding:8px 10px; border-radius:12px;">{weather_selector.value}</div>
      </div>
      <div style="display:grid; grid-template-columns:1fr 1fr; gap:10px;">
        <div>📏 <b>Quãng đường</b><br><span>{round(total_distance/1000,2)} km</span></div>
        <div>💰 <b>Phí cước</b><br><span>{format_vnd(final_fare)} VND</span></div>
        <div>⏱ <b>Chờ đón</b><br><span>{round(wait_time_min,1)} phút</span></div>
        <div>🧭 <b>Tổng thời gian</b><br><span>{round(total_time_min,1)} phút</span></div>
      </div>
      <div style="margin-top:10px; font-size:12px; opacity:.8;">Giá đã bao gồm các yếu tố giao thông và thời tiết theo thời điểm hiện tại (mô phỏng).</div>
      <div style="margin-top:12px; display:flex; gap:10px;">
        <button style="background:{PRIMARY}; color:white; border:none; padding:10px 12px; border-radius:10px; cursor:pointer;">Đặt xe ngay</button>
        <button style="background:#e2e8f0; color:#111827; border:none; padding:10px 12px; border-radius:10px; cursor:pointer;">Chia sẻ</button>
      </div>
    </div>
    """
    fare_display.value = fare_card_html

    # Vẽ map (folium)
    m = folium.Map(location=start_coords, zoom_start=13, tiles='cartodbpositron')

    # Điểm đi/đến
    folium.Marker(start_coords, popup=f"🏁 Điểm đi: {start_name}", tooltip=f"Điểm đi: {start_name}",
                  icon=folium.Icon(color='blue', icon='play', prefix='fa')).add_to(m)
    folium.Marker(end_coords, popup=f"🎯 Điểm đến: {end_name}", tooltip=f"Điểm đến: {end_name}",
                  icon=folium.Icon(color='red', icon='flag-checkered', prefix='fa')).add_to(m)

    # Đường xe → điểm đón, và từ điểm đi → điểm đến
    try:
        folium.PolyLine(coords_to_start, color=PRIMARY, weight=5, opacity=0.9).add_to(m)
        folium.PolyLine(coords_start_to_end, color=SECONDARY, weight=5, opacity=0.9).add_to(m)
    except Exception:
        pass

    # Hiển thị các điểm POI
    for name, coord in tourist_spots.items():
        folium.CircleMarker(coord, radius=4, color='#6b7280', fill=True, fill_opacity=0.8, tooltip=name).add_to(m)

    # Các xe xung quanh
    for v in vehicle_data:
        color = 'green' if v == nearest_vehicle else ('gray' if v['type'] == 'Ô tô' else 'orange')
        icon_name = 'car' if v['type'] == 'Ô tô' else 'motorcycle'
        folium.Marker((v['lat'], v['lon']), popup=f"{v['type']}",
                      icon=folium.Icon(color=color, icon=icon_name, prefix='fa')).add_to(m)

    # Render giao diện
    clear_output(wait=True)
    display(HTML(APP_HEADER))
    display(alert_box)
    display(HTML(CARD_WRAP_START))
    display(start_selector)
    display(end_selector)
    display(vehicle_selector)
    display(traffic_selector)
    display(weather_selector)
    buttons_box = widgets.HBox([compute_button, reset_button])
    display(buttons_box)
    display(HTML(CARD_WRAP_MID_END))
    display(fare_display)
    display(m)

# ====================== Sự kiện nút bấm ======================

def on_reset(_):
    start_selector.value = list(tourist_spots.keys())[0]
    end_selector.value = list(tourist_spots.keys())[1]
    vehicle_selector.value = 'Ô tô'
    traffic_selector.value = list(traffic_levels.keys())[2]  # Trung bình
    weather_selector.value = list(weather_levels.keys())[0]  # Trời đẹp
    fare_display.value = "<i>Chưa có dữ liệu. Hãy chọn điểm đi/đến và bấm tính.</i>"
    alert_box.value = ""

compute_button.on_click(compute_and_display)
reset_button.on_click(on_reset)

# ====================== Hiển thị UI lần đầu ======================
display(HTML(APP_HEADER))
display(alert_box)
display(HTML(CARD_WRAP_START))
display(start_selector)
display(end_selector)
display(vehicle_selector)
display(traffic_selector)
display(weather_selector)
buttons_box = widgets.HBox([compute_button, reset_button])
display(buttons_box)
display(HTML(CARD_WRAP_MID_END))
display(fare_display)


HTML(value='')

Dropdown(description='Điểm đi:', layout=Layout(width='100%'), options=('Chợ Bến Thành', 'Dinh Độc Lập', 'Nhà t…

Dropdown(description='Điểm đến:', layout=Layout(width='100%'), options=('Chợ Bến Thành', 'Dinh Độc Lập', 'Nhà …

ToggleButtons(button_style='success', description='Phương tiện:', options=(('🚗 Ô tô', 'Ô tô'), ('🛵 Xe máy', 'X…

RadioButtons(description='Giao thông:', options=('Rất thấp 🌐 ', 'Thấp 📢', 'Trung bình 🚦', 'Cao 🚧', 'Rất cao ‼️…

RadioButtons(description='Thời tiết:', options=('Trời đẹp ☀️', 'Ít mưa 🌦️', 'Mưa 🌧️', 'Mưa lớn ⛈️', 'Bão 🌫'), …

HTML(value='<i>Chưa có dữ liệu. Hãy chọn điểm đi/đến và bấm tính.</i>')